# Cost Tracking

В данный момент один общий трекер для всех оптимизаторов, поэтому если хочется получить статистику по для каждого запуска отдельно, то нужно сбрасывать её

In [1]:
import os
from getpass import getpass

api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API key: ")

from coolprompt.language_model import create_chat_model
from coolprompt.assistant import PromptTuner
from datasets import load_dataset


/home/asd480/anaconda3/envs/coolprompt/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model = create_chat_model(
    model="gpt-4o-mini",
    temperature=0.7,
    max_tokens=4000,
)

tuner = PromptTuner(target_model=model, system_model=model)


[2025-12-25 21:57:14,067] [INFO] [assistant.__init__] - Validating the target model
[2025-12-25 21:57:14,067] [INFO] [assistant.__init__] - PromptTuner successfully initialized


In [4]:
samsum = load_dataset("knkarthick/samsum")
dataset = samsum["train"]["dialogue"][:10]
targets = samsum["train"]["summary"][:10]


In [5]:
tuner.reset_stats()

initial_stats = tuner.get_stats()
print("Initial stats:", initial_stats)


Initial stats: {'total_calls': 0, 'total_tokens': 0, 'prompt_tokens': 0, 'completion_tokens': 0, 'total_cost': 0.0, 'invoke_calls': 0, 'batch_calls': 0, 'batch_items': 0}


In [6]:
final_prompt = tuner.run(
    start_prompt="Summarize the dialogue",
    task="generation",
    dataset=dataset,
    target=targets,
    method="reflective",
    metric="meteor",
    population_size=5,
    num_epochs=3,
    verbose=1,
)


[2025-12-25 21:57:19,972] [INFO] [assistant.run] - Validating args for PromptTuner running
[nltk_data] Downloading package wordnet to /home/asd480/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/asd480/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/asd480/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[2025-12-25 21:57:21,786] [INFO] [evaluator.__init__] - Evaluator successfully initialized with meteor metric
[2025-12-25 21:57:25,552] [INFO] [assistant.run] - === Starting Prompt Optimization ===
[2025-12-25 21:57:25,553] [INFO] [assistant.run] - Method: reflective, Task: generation
[2025-12-25 21:57:25,553] [INFO] [assistant.run] - Metric: meteor, Validation size: 0.25
[2025-12-25 21:57:25,554] [INFO] [assistant.run] - Dataset: 10 samples
[2025-12-25 21:57:25,554] [INFO] [assistant.run] - Target: 10 samples
[2025-12-25 21:57:25,55

In [7]:
stats = tuner.get_stats()

if stats:
    print("cost tracking")
    print(f"total calls {stats['total_calls']}")
    print(f"invoke calls {stats['invoke_calls']}")
    print(f"batch calls {stats['batch_calls']}")
    print(f"batch items {stats['batch_items']}")
    print()
    print(f"total tokens {stats['total_tokens']:,}")
    print(f"prompt tokens {stats['prompt_tokens']:,}")
    print(f"completion tokens {stats['completion_tokens']:,}")
    print()
    print(f"total cost: ${stats['total_cost']:.6f}")


cost tracking
total calls 59
invoke calls 5
batch calls 54
batch items 312

total tokens 99,890
prompt tokens 73,723
completion tokens 26,167

total cost: $0.026759


In [8]:
print(f"Initial metric: {tuner.init_metric:.4f}")
print(f"Final metric: {tuner.final_metric:.4f}")
print(f"\nInitial prompt: {tuner.init_prompt}")
print(f"\nFinal prompt: {tuner.final_prompt}")


Initial metric: 0.4185
Final metric: 0.4847

Initial prompt: Summarize the dialogue

Final prompt: Please extract and summarize the key points and conclusions from the dialogue, focusing on clarity and coherence while omitting extraneous details to facilitate easier understanding and future reference.
